<img src="image.jpg">

 # PyPRT - Building Modeling Optimization

This notebook is used to show how the Python bindings for PRT can be employed in optimizing the rule attributes of a generated model to achive predefined performance goals.

#### Imports and paths-related functions

In [1]:
import sys, os

sys.path.append(os.path.join(os.getcwd(), "..", "src"))
from utility import visualize_PRT_results, combine_reports

SDK_PATH = os.path.join(os.getcwd(), "..", "install", "bin")
sys.path.append(SDK_PATH)

import pyprt

from scipy import optimize
import numpy as np
import time

CS_FOLDER = os.getcwd()
def asset_file(filename):
    return os.path.join(CS_FOLDER, filename)

from IPython.display import IFrame
from arcgis import GIS

In [2]:
my_gis = GIS(username="CLechot_zurich")

Enter password: ········


In [3]:
print("\nInitializing PRT.")
pyprt.initialize_prt(SDK_PATH)

if not pyprt.is_prt_initialized():
    raise Exception("PRT is not initialized")


Initializing PRT.
[PRT] [2019-10-03 09:20:01] [info] Esri Procedural Runtime
[PRT] [2019-10-03 09:20:01] [info] Version: 2.1.5704 | Build Date: Tue 09/03/2019 06:40 PM | Build Config: PRT_BC_REL PRT_CC_OPT PRT_TC_VC141 | OS: win32 | ARCH: x86_64
[PRT] [2019-10-03 09:20:01] [debug] prtPlugins item 0: 'C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\..\install\lib'
[PRT] [2019-10-03 09:20:01] [error] failed to load library: error while loading library 'C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\..\install\lib\com.esri.prt.unreal.dll': The specified module could not be found.

[PRT] [2019-10-03 09:20:01] [warning] failed to load library com.esri.prt.unreal.dll
[PRT] [2019-10-03 09:20:01] [error] failed to load library: error while loading library 'C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\..\install\lib\DatasmithSDK.dll': The specified module could not 

## Optimization Problem Definition

In this notebook, we are using a CGA rule that allows us to generate modern and sustainable buildings. The architecture of these buildings is focused on the green aspect of the building. Therefore, in addition to be modern and esthetical, the generated building answers also to the actual requirements of the city which are to encourage the integration of greenery on the buildings.

<img src="green_building_parameters.png" width="1000" height="100">

In [4]:
rpk_green = asset_file("envelope0110.rpk")
attrs_green = {'ruleFile' : "rules/typology/envelope0110.cgb", 'startRule' : "Default$Lot", 'report_but_not_display_green' : True}

shape_geo = asset_file("parcelN.obj")

### Optimization with envelope.rpk

Step 1 : optimize the green area based on the lot coverage parameter

### The variables

In [5]:
goal_str_green = "Greenery Area" + "_sum"

input_attr_green = ["lot_coverage_parameter"]
x0_green = [float(50)]

In [6]:
m = pyprt.ModelGenerator(shape_geo)
model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.examples.PyEncoder", {'emitGeometry' : False})

[PRT] [2019-10-03 09:20:08] [debug] trying to read initial shape geometry from file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5CparcelN.obj

[PRT] [2019-10-03 09:20:08] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope0110.rpk

[PRT] [2019-10-03 09:20:08] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/vertical_greenery/1.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/vertical_greenery/1.PNG" type="str"/>
	<attribute key="assets/vertical_greenery/6.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/vertical_greenery/6.PNG" type="str"/>
	<attribute key="assets/lawn2.jpg" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDoc

In [7]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    generated_mod_green = m.generate_model(attrs_green)
    
    if generated_mod_green:
        rep_green = combine_reports(generated_mod_green[0])
        return -rep_green[goal_str_green]
    else:
        print("Error in optimization process.")

In [8]:
bounds_green = [(0,100)]

In [9]:
start = time.time()

res_green = optimize.differential_evolution(fct_green, bounds_green, seed = 666)

duration = time.time() - start
print("The optimization lasted: " + str(duration))
res_green

The optimization lasted: 16.442068338394165


     fun: -2196.7870170993974
 message: 'Optimization terminated successfully.'
    nfev: 32
     nit: 1
 success: True
       x: array([82.26263821])

### Results Visualization

In [16]:
export_file_name = "Demo_SLPKExport1"
enc_optionsSLPK = {'sceneType' : "Local",'baseName' : export_file_name,'sceneWkid' : "3857",'layerTextureEncoding' : ["2"],'layerEnabled' : [True],'layerUID' : ["1"],'layerName' : ["1"],'layerTextureQuality' : [1.0],'layerTextureCompression' : [9],'layerTextureScaling' : [1.0],'layerTextureMaxDimension' : [2048],'layerFeatureGranularity' : ["0"],'layerBackfaceCulling' : [False]}

attrs_green[input_attr_green[0]] = res_green.x[0]
green_model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.codecs.I3SEncoder", enc_optionsSLPK)

[PRT] [2019-10-03 09:25:28] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope0110.rpk

[PRT] [2019-10-03 09:25:28] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/vertical_greenery/1.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/vertical_greenery/1.PNG" type="str"/>
	<attribute key="assets/vertical_greenery/6.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/vertical_greenery/6.PNG" type="str"/>
	<attribute key="assets/lawn2.jpg" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/lawn2.jpg" type="str"/>
	<attribute key="assets/Bench.mtl" value="rpk:file:/C:%5CUsers%5Ccami9495%5C

[PRT] [2019-10-03 09:25:29] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-10-03 09:25:29] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:25:29] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:25:29] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-10-03 09:25:29] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:25:29] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-10-03 09:25:29] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:25:29] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:25:29] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-10-03 09:25:29] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:25:29] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-10-03 09:25:31] [debug] width 1024, height 1024, rasterCount 3
[PRT] [2019-10-03 09:25:31] [debug] width 1024, height 1024, rasterCount 3
[PRT] [2019-10-03 09:25:31] [debug] width 1024,

In [17]:
item = my_gis.content.add({'type':'Scene Package'}, data=asset_file("..\output\%s.slpk" % export_file_name), folder='demo')
published_item = item.publish()

In [19]:
published_item

<Item title:"Demo_SLPKExport1" type:Scene Layer owner:CLechot_zurich>

In [18]:
source = 'http://zurich.maps.arcgis.com/home/webscene/viewer.html?layers=' + published_item.id #'5e1dc57819e94457bae607dd3551e274'
IFrame(src=source, width=900, height=600)

-------------------------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------------------

Step 2 : optimize the total green area reported including the balconies trees. This will be optimized taking into account several parameters:
- the shape of the building (1 = orthogonal, 2 = L shape, 3 = U shape, 4 = O shapes)
- the lot coverage
- the height of the first tier

### The variables

In [26]:
goal_str_green = "Greenery Area" + "_sum"

input_attr_green = ["lot_coverage_parameter", "height_first_tier", "shape_of_building"]
x0_green = [float(50), float(9), float(round(2))]

In [27]:
attrs_green = {'ruleFile' : "rules/typology/envelope0110.cgb", 'startRule' : "Default$Lot", 'report_but_not_display_green' : True}

m = pyprt.ModelGenerator(shape_geo)
model2 = m.generate_model(attrs_green, rpk_green, "com.esri.prt.examples.PyEncoder", {'emitGeometry' : False})

[PRT] [2019-10-03 09:32:02] [debug] trying to read initial shape geometry from file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5CparcelN.obj

[PRT] [2019-10-03 09:32:02] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope0110.rpk

[PRT] [2019-10-03 09:32:02] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/wood.jpg" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/wood.jpg" type="str"/>
	<attribute key="/green_optimization_3/assets/vertical_greenery/1.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/vertical_greenery/1.PNG" type="str"/>
	<attribute key="/green_optimization_3/rules/typology/ground_floor_plan.cga" value="rpk

In [28]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    attrs_green[input_attr_green[1]] = x[1]
    attrs_green[input_attr_green[2]] = float(round(x[2]))
    generated_mod_green = m.generate_model(attrs_green)
    
    if generated_mod_green:
        rep_green = combine_reports(generated_mod_green[0])
        return -rep_green[goal_str_green]
    else:
        print("Error in optimization process.")

In [29]:
bounds_green = [(0,100),(8,13),(1,4)]

In [31]:
start = time.time()

res_green = optimize.differential_evolution(fct_green, bounds_green, tol = 0.15, seed = 666)

duration = time.time() - start
print("The optimization lasted: " + str(duration))
res_green

The optimization lasted: 103.1142783164978


     fun: -4065.8768645564187
 message: 'Optimization terminated successfully.'
    nfev: 184
     nit: 3
 success: True
       x: array([34.99030757, 11.35376576,  3.69885958])

### Results Visualization

In [32]:
export_file_name = "Demo_SLPKExport2"
enc_optionsSLPK = {'sceneType' : "Local",'baseName' : export_file_name,'sceneWkid' : "3857",'layerTextureEncoding' : ["2"],'layerEnabled' : [True],'layerUID' : ["1"],'layerName' : ["1"],'layerTextureQuality' : [1.0],'layerTextureCompression' : [9],'layerTextureScaling' : [1.0],'layerTextureMaxDimension' : [2048],'layerFeatureGranularity' : ["0"],'layerBackfaceCulling' : [False]}

attrs_green[input_attr_green[0]] = res_green.x[0]
attrs_green[input_attr_green[1]] = res_green.x[1]
attrs_green[input_attr_green[2]] = float(round(res_green.x[2]))
green_model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.codecs.I3SEncoder", enc_optionsSLPK)

[PRT] [2019-10-03 09:39:49] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope0110.rpk

[PRT] [2019-10-03 09:39:49] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/wood.jpg" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/wood.jpg" type="str"/>
	<attribute key="/green_optimization_3/assets/vertical_greenery/1.PNG" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/vertical_greenery/1.PNG" type="str"/>
	<attribute key="/green_optimization_3/rules/typology/ground_floor_plan.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/rules/typology/ground_floor_plan.cga" type="str"/>
	<attribute key="/gree

[PRT] [2019-10-03 09:39:52] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-10-03 09:39:52] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:39:52] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-10-03 09:39:52] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:39:52] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:39:52] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-10-03 09:39:52] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:39:52] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-10-03 09:39:53] [debug] width 1024, height 1024, rasterCount 3
[PRT] [2019-10-03 09:39:54] [debug] width 1024, height 1024, rasterCount 3
[PRT] [2019-10-03 09:39:54] [debug] width 1024, height 1024, rasterCount 3
[PRT] [2019-10-03 09:39:54] [debug] width 289, height 490, rasterCount 4
[PRT] [2019-10-03 09:39:54] [debug] width 490, height 737, rasterCount 4
[PRT] [2019-10-03 09:39:54] [debug] width 302

In [33]:
item = my_gis.content.add({'type':'Scene Package'}, data=asset_file("..\output\%s.slpk" % export_file_name), folder='demo')
published_item = item.publish()

In [35]:
published_item

<Item title:"Demo_SLPKExport2" type:Scene Layer owner:CLechot_zurich>

In [34]:
source2 = 'http://zurich.maps.arcgis.com/home/webscene/viewer.html?layers=' + published_item.id #'83ecc94cc2804b0aaeb235e46ce4be14'
IFrame(src=source2, width=900, height=600)

-------------------------------------------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------------------------------------------

Step 3 : optimize the total green area reported including the balconies trees AND the floor area in order to avoid thin walls. We will keep O-shape building. This will be optimized taking into account several parameters:

- the lot coverage
- the height of the first tier

### The variables

In [36]:
goal_str_green = "Greenery Area" + "_sum"
goal_str_floor = "Floor area" + "_sum"
floor_weight = 0.07 #this allows having similar range between the green area and the floor area for a building with default values and 100% lot coverage --> green area / floor area = 1499/19445=0.077

input_attr_green = ["lot_coverage_parameter", "height_first_tier"]
x0_green = [float(50), float(9)]

In [37]:
attrs_green = {'ruleFile' : "rules/typology/envelope0110.cgb", 'startRule' : "Default$Lot", 'report_but_not_display_green' : True, 'shape_of_building' : 4.0}

m = pyprt.ModelGenerator(shape_geo)
model3 = m.generate_model(attrs_green, rpk_green, "com.esri.prt.examples.PyEncoder", {'emitGeometry' : False})

[PRT] [2019-10-03 09:41:42] [debug] trying to read initial shape geometry from file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5CparcelN.obj

[PRT] [2019-10-03 09:41:42] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope0110.rpk

[PRT] [2019-10-03 09:41:42] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/bench.obj" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/bench.obj" type="str"/>
	<attribute key="/ESRI.lib/rules/Plants/Plant_Loader.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/.ws/ESRI.lib/rules/Plants/Plant_Loader.cga" type="str"/>
	<attribute key="/green_optimization_3/assets/glass_door.obj" value="rpk:file:/C:%5CUs

In [38]:
def fct_green(x):

    attrs_green[input_attr_green[0]] = x[0]
    attrs_green[input_attr_green[1]] = x[1]
    generated_mod_green = m.generate_model(attrs_green)
    
    if generated_mod_green:
        rep_green = combine_reports(generated_mod_green[0])
        return -(rep_green[goal_str_green]+floor_weight*rep_green[goal_str_floor])
    else:
        print("Error in optimization process.")

In [39]:
bounds_green = [(0,100),(8,13)]

In [41]:
start = time.time()

res_green = optimize.differential_evolution(fct_green, bounds_green, tol = 0.1, seed = 666)

duration = time.time() - start
print("The optimization lasted: " + str(duration))
res_green

The optimization lasted: 87.6885027885437


     fun: -4702.021447917708
 message: 'Optimization terminated successfully.'
    nfev: 93
     nit: 2
 success: True
       x: array([60.56927098, 12.64878533])

### Results Visualization

In [42]:
export_file_name = "Demo_SLPKExport3"
enc_optionsSLPK = {'sceneType' : "Local",'baseName' : export_file_name,'sceneWkid' : "3857",'layerTextureEncoding' : ["2"],'layerEnabled' : [True],'layerUID' : ["1"],'layerName' : ["1"],'layerTextureQuality' : [1.0],'layerTextureCompression' : [9],'layerTextureScaling' : [1.0],'layerTextureMaxDimension' : [2048],'layerFeatureGranularity' : ["0"],'layerBackfaceCulling' : [False]}

attrs_green[input_attr_green[0]] = res_green.x[0]
attrs_green[input_attr_green[1]] = res_green.x[1]
green_model = m.generate_model(attrs_green, rpk_green, "com.esri.prt.codecs.I3SEncoder", enc_optionsSLPK)

[PRT] [2019-10-03 09:48:00] [info] using rule package C:\Users\cami9495\Documents\esri-cityengine-sdk-master\examples\py4prt\Demo2_Optimization\envelope0110.rpk

[PRT] [2019-10-03 09:48:00] [debug] resolve map = <ResolveMap>
	<attribute key="/green_optimization_3/assets/bench.obj" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/bench.obj" type="str"/>
	<attribute key="/ESRI.lib/rules/Plants/Plant_Loader.cga" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/.ws/ESRI.lib/rules/Plants/Plant_Loader.cga" type="str"/>
	<attribute key="/green_optimization_3/assets/glass_door.obj" value="rpk:file:/C:%5CUsers%5Ccami9495%5CDocuments%5Cesri-cityengine-sdk-master%5Cexamples%5Cpy4prt%5CDemo2_Optimization%5Cenvelope0110.rpk!/assets/glass_door.obj" type="str"/>
	<attribute key="/green_optimization_3/assets/wall_

[PRT] [2019-10-03 09:48:02] [debug] width 489, height 489, rasterCount 3
[PRT] [2019-10-03 09:48:02] [debug] width 850, height 850, rasterCount 3
[PRT] [2019-10-03 09:48:04] [debug] width 1024, height 1024, rasterCount 3
[PRT] [2019-10-03 09:48:04] [debug] width 1024, height 1024, rasterCount 3
[PRT] [2019-10-03 09:48:04] [debug] width 1024, height 1024, rasterCount 3
[PRT] [2019-10-03 09:48:04] [debug] width 327, height 490, rasterCount 4
[PRT] [2019-10-03 09:48:04] [debug] width 325, height 490, rasterCount 4
[PRT] [2019-10-03 09:48:04] [debug] width 302, height 490, rasterCount 4
[PRT] [2019-10-03 09:48:04] [debug] width 489, height 662, rasterCount 4
[PRT] [2019-10-03 09:48:04] [debug] width 490, height 662, rasterCount 4
[PRT] [2019-10-03 09:48:04] [debug] width 289, height 490, rasterCount 4
[PRT] [2019-10-03 09:48:04] [debug] width 489, height 733, rasterCount 4
[PRT] [2019-10-03 09:48:04] [debug] width 490, height 689, rasterCount 4
[PRT] [2019-10-03 09:48:04] [debug] width 327

In [43]:
item = my_gis.content.add({'type':'Scene Package'}, data=asset_file("..\output\%s.slpk" % export_file_name), folder='demo')
published_item = item.publish()

In [44]:
published_item

<Item title:"Demo_SLPKExport3" type:Scene Layer owner:CLechot_zurich>

In [45]:
source3 = 'http://zurich.maps.arcgis.com/home/webscene/viewer.html?layers=' + published_item.id #'f3f78a9607814079a85c045197b092f3'
IFrame(src=source3, width=900, height=600)

In [46]:
print("\nShutdown PRT.")
pyprt.shutdown_prt()


Shutdown PRT.
